# Volunteer Certificate
reads data from CSV, dynamically resizes text to fit bounding box, then applies text style, and outputs image as pdf

# Features

1. The script loads an empty image template (e.g. a **.png** file), then reads data from csv file (columns: **name**, **start**, **end**). The **start** and **end** columns are the year the volunteer joined and left respectively.

2. Dynamic text-resizing: If a volunteer has a very long name, its font size will decrease gradually until the text is able to fit in the bounding box.

3. 1-year term and multi-year terms: If a volunteer joined and left in the same year (i.e. in the csv file,  `start == 2020`, `end == 2020 ` ), then on the certificate, only that year will appear in the lower right corner (e.g. `2020`). Otherwise, on the certificate the year will appear as a range (e.g. `2016 - 2020`).

4. Output file as **name.pdf**, where the **name** is the volunteer name. For convenience a zip file of all pdf outputs is also generated.

5. Unique file names: In a scenario where volunteers have the same name, the output file for the 2nd volunteer will say **name_1.pdf** (where **name** is the name of the volunteer), and for the 3rd volunteer it will say **name_2.pdf**, etc. In this you don't have to worry about overwriting the previous files.


# Notes

1. Example Image info
 - size: (1790, 1276)
 - 150 dpi
 - RGB


2. **Visualize the bounding box for the name area**

```Python
x1, y1, x2, y2=[298.2, 484, 1512.3, 660]
name_box=draw.rectangle([x1, y1, x2, y2], outline='red')
image
```
Recommended max size for text in order to fit into this bounding box:
- max width for text:  ~ 1120
- max height for text: ~ 200

If using *Rochester-Regular* font type, a font size of 150 is generally good starting point.

## Code

In [1]:
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont
import pandas as pd
import shutil
import os

In [11]:
def getSize(draw, info, max_w, font_size, font_style='Rochester-Regular.otf'):
    myFont=ImageFont.truetype(font_style, font_size)
    w, h=draw.textsize(info, font=myFont)

    while w > max_w: #recommended max width, see note above
        font_size-=1
        myFont=ImageFont.truetype(font_style, font_size)
        w, h=draw.textsize(info, font=myFont)
    return (w, h, myFont)

def writeInfo(draw, info, box, max_w, font_size):

    #Coordinates of bounding box of name area
    x1, y1, x2, y2=box

    #Dynamically optimize font size to fit in the bounding box for name
    w, h, myFont=getSize(draw, info, max_w, font_size)

    #Add text with optimized font size
    draw.text([(x1+x2-w)/2, (y1+y2-h)/2],info,
                   align='center',
                   fill='#43729F', font=myFont)

def getCert(name, year):

    #Load template
    image=Image.open("Certificate_Demo.png").convert("RGB")
    draw=ImageDraw.Draw(image)

    name_box=[298.2, 484, 1512.3, 660]
    max_w_name=1120
    writeInfo(draw, name, name_box, max_w_name, font_size=150)

    year_box=[1165, 950, 1516, 1042]
    max_w_year=308
    writeInfo(draw, year, year_box, max_w_year, font_size=60)

    #return image object
    return image


In [12]:
def getUniqueFn(name, unique_fn):
    ct=1
    while name in unique_fn:
        n=name.split('-')[0]  #get the name portion only
        name=n+'-'+ str(ct)
        ct+=1
    unique_fn.add(name)
    return name

def certicate(data):

    unique_fn=set([])

    for i in range (0, data.shape[0]):
        name, startYear, endYear=data.iloc[i, 0], data.iloc[i, 1], data.iloc[i, 2]
        startYear, endYear=str(startYear), str(endYear)
        if startYear==endYear:
            image=getCert(name, str(endYear))
            fn='result/'+getUniqueFn(name, unique_fn) +'.pdf'  #get unique file name
            image.save(fn)
        else:
            image=getCert(name, str(startYear)+' - '+str(endYear))
            fn='result/'+getUniqueFn(name, unique_fn) +'.pdf'  #get unique file name
            image.save(fn)

In [13]:
def makeDir():
    folderPath="result"
    try:
        shutil.rmtree(folderPath)  #delete 'result' folder and all its current contents if it exist
    except:
        pass  #if it doesn't exist then don't do anything
    os.mkdir(folderPath) #make a new `result` folder
    


In [15]:
if __name__ =='__main__':
    makeDir()
    data=pd.read_csv("volunteers.csv")
    certicate(data)
    shutil.make_archive("result", 'zip', "result") #zip up file

'/Users/mightycamole/Desktop/Data Science/Python/WorkWeb/CAS_Github/general-image-processing/volunteerCertificate'